In [2]:
import pandas as pd
import requests 
import json
import os
import mysql.connector

In [9]:

# Extracting data from cloning to github
path="data/aggregated/transaction/country/india/state/"
states_list = os.listdir(path)  #to direct the path and get list of states data
#states_list

In [10]:
#Aggregated Transactions
file = {"State":[],"Year":[],"Quater":[],"Transaction_type":[],"Transaction_count":[],"Transaction_amount":[]}
for i in states_list:                        
    state_i = path+i+"/"                                 #get the path state-wise to see the year
    state_yr = os.listdir(state_i)
    for j in state_yr:
        state_yr_sum = state_i+j+"/"                     #get the path yearwise for respective state to see how many quarters
        state_qtr = os.listdir(state_yr_sum)
        for k in state_qtr:    
            state_k = state_yr_sum+k                     
            datas = open(state_k,"r")                    # open the json format of each quarter
            form = json.load(datas)
            for l in form["data"]["transactionData"]:
                Name = l["name"]
                Count = l["paymentInstruments"][0]["count"]
                Amount =l["paymentInstruments"][0]["amount"]
                file["Transaction_type"].append(Name)
                file["Transaction_count"].append(Count)
                file["Transaction_amount"].append(Amount)
                file["State"].append(i)
                file["Year"].append(j)
                file["Quater"].append(int(k.strip(".json")))

phonepe = pd.DataFrame(file)
phonepe

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,6.202222e+11
3590,west-bengal,2022,4,Merchant payments,171667404,1.408077e+11
3591,west-bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10
3592,west-bengal,2022,4,Financial Services,268388,2.611229e+08


In [14]:
#cleaning and pre-processing 
ph1_df = phonepe.copy()
ph1_df["Year"] = pd.to_datetime(ph1_df["Year"])

In [15]:
ph1_df["Year"] = ph1_df["Year"].dt.year
ph1_df["Year"].dtype
ph1_df["Transaction_amount"] = ph1_df["Transaction_amount"].astype(int)
ph1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   State               3594 non-null   object
 1   Year                3594 non-null   int64 
 2   Quater              3594 non-null   int64 
 3   Transaction_type    3594 non-null   object
 4   Transaction_count   3594 non-null   int64 
 5   Transaction_amount  3594 non-null   int32 
dtypes: int32(1), int64(3), object(2)
memory usage: 154.6+ KB


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [16]:
ph1_df.drop_duplicates()

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1845307
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,12138655
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,452507
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,10601
4,andaman-&-nicobar-islands,2018,1,Others,256,184689
...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,-2147483648
3590,west-bengal,2022,4,Merchant payments,171667404,-2147483648
3591,west-bengal,2022,4,Recharge & bill payments,48921147,-2147483648
3592,west-bengal,2022,4,Financial Services,268388,261122865


In [17]:
#agg_user
path1="data/aggregated/user/country/india/state/"
states_list1 = os.listdir(path1)
states_list1

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']

In [18]:
file1 =  {"State":[],"Year":[],"Quater":[],"Brand":[],"Brand_count":[],"Brand_percentage":[]}
for i in states_list1:                        
    ustate_i = path1+i+"/"
    ustate_yr = os.listdir(ustate_i)
    for j in ustate_yr:
        ustate_yr_sum = ustate_i+j+"/"                     #get the path yearwise for respective state to see how many quarters
        ustate_qtr = os.listdir(ustate_yr_sum)
        for k in ustate_qtr:
            ustate_k = ustate_yr_sum+k                     
            udatas = open(ustate_k,"r")                    # open the json format of each quarter
            uform = json.load(udatas)
            try:
                for z in uform['data']['usersByDevice']:
                    Brand = z['brand']
                    Brand_count = z['count']
                    Brand_percentage = z['percentage']
                    file1["Brand"].append(Brand)
                    file1["Brand_count"].append(Brand_count)
                    file1["Brand_percentage"].append(Brand_percentage)
                    file1["State"].append(i)
                    file1["Year"].append(j)
                    file1["Quater"].append(int(k.strip('.json'))) 
            except:
                pass

userDetail_phonepe = pd.DataFrame(file1)
userDetail_phonepe.head()

,State,Year,Quater,Brand,Brand_count,Brand_percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258


In [19]:
ud_df = userDetail_phonepe.copy()
ud_df["Year"]=pd.to_datetime(ud_df["Year"])
ud_df["Year"]=ud_df["Year"].dt.year
ud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             6732 non-null   object 
 1   Year              6732 non-null   int64  
 2   Quater            6732 non-null   int64  
 3   Brand             6732 non-null   object 
 4   Brand_count       6732 non-null   int64  
 5   Brand_percentage  6732 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.7+ KB


In [20]:
#map_transaction
path2 = "data/map/transaction/hover/country/india/state/"
states_list2 = os.listdir(path2)
states_list2

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']

In [21]:
file2 = {'State': [], 'Year': [], 'Quater': [], 'District': [],'Transaction_count': [], 'Transaction_amount': [] }
for i in states_list2:
    tstate_i = path2+i+"/"
    tstate_yr = os.listdir(tstate_i)
    for j in tstate_yr:
        tstate_yr_sum = tstate_i+j+"/"
        tstate_qtr = os.listdir(tstate_yr_sum)
        for k in tstate_qtr:
            tstate_k = tstate_yr_sum+k
            tdatas = open(tstate_k,"r")
            tform = json.load(tdatas)
            try:
                for z in tform['data']['hoverDataList']:
                    district = z['name']
                    transaction_count = z['metric'][0]['count']
                    transaction_amount = z['metric'][0]['amount']
                    file2['District'].append(district)
                    file2['Transaction_count'].append(transaction_count)
                    file2['Transaction_amount'].append(transaction_amount)
                    file2["State"].append(i)
                    file2["Year"].append(j)
                    file2["Quater"].append(int(k.strip('.json'))) 
            except:
                pass

map_transaction = pd.DataFrame(file2)
map_transaction

,State,Year,Quater,District,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
14631,west-bengal,2022,4,nadia district,12690126,2.804568e+10
14632,west-bengal,2022,4,birbhum district,7617444,1.614650e+10
14633,west-bengal,2022,4,purba medinipur district,14484229,3.309949e+10
14634,west-bengal,2022,4,maldah district,12492746,2.721861e+10


In [22]:
mt_df = map_transaction.copy()
mt_df["Year"] = pd.to_datetime(mt_df["Year"])
mt_df["Year"]=mt_df["Year"].dt.year
mt_df["Transaction_amount"] = mt_df["Transaction_amount"].astype(int)
mt_df.head()
mt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14636 entries, 0 to 14635
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   State               14636 non-null  object
 1   Year                14636 non-null  int64 
 2   Quater              14636 non-null  int64 
 3   District            14636 non-null  object
 4   Transaction_count   14636 non-null  int64 
 5   Transaction_amount  14636 non-null  int32 
dtypes: int32(1), int64(3), object(2)
memory usage: 629.0+ KB


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [23]:
#map_user
path3 = "data/map/user/hover/country/india/state/"
states_list3 = os.listdir(path3)
states_list3

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']

In [24]:
file3 ={'State':[], 'Year':[],'Quater':[],"District":[],'Registered_user':[],"App_opening":[]}

for i in states_list3:
    ustate_i = path3+i+"/"
    ustate_yr = os.listdir(ustate_i)
    for j in ustate_yr:
        ustate_yr_sum = ustate_i+j+"/"
        ustate_qtr = os.listdir(ustate_yr_sum)
        for k in ustate_qtr:
            ustate_k = ustate_yr_sum+k
            udatas = open(ustate_k, 'r')
            uform= json.load(udatas)
            try:
                for z in uform['data']["hoverData"]:
                    district = z
                    registered_user =  uform['data']["hoverData"][z]["registeredUsers"]
                    app_opening = uform['data']["hoverData"][z]["appOpens"]
                    file3['District'].append(district)
                    file3['Registered_user'].append(registered_user)
                    file3['App_opening'].append(app_opening)
                    file3['State'].append(i)
                    file3['Year'].append(j)
                    file3['Quater'].append(int(k.strip('.json')))
            except:
                pass

district_registers = pd.DataFrame(file3)
district_registers

,State,Year,Quater,District,Registered_user,App_opening
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
14635,west-bengal,2022,4,nadia district,1359420,33853990
14636,west-bengal,2022,4,birbhum district,855236,20950662
14637,west-bengal,2022,4,purba medinipur district,1346908,38278506
14638,west-bengal,2022,4,maldah district,954892,29023743


In [25]:
dt_df = district_registers.copy()
dt_df["Year"] = pd.to_datetime(dt_df["Year"])
dt_df["Year"] = dt_df["Year"].dt.year
dt_df.head()
dt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            14640 non-null  object
 1   Year             14640 non-null  int64 
 2   Quater           14640 non-null  int64 
 3   District         14640 non-null  object
 4   Registered_user  14640 non-null  int64 
 5   App_opening      14640 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 686.4+ KB


In [26]:
#top_transaction
path4 = "data/top/transaction/country/india/state/"
states_list4 = os.listdir(path4)
states_list4

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']

In [27]:
file4 = {"State":[],"Year":[],"Quater":[],"District":[],"Transaction_count":[],"Transaction_amount":[],"Pincode":[]}
for i in states_list4:
    tstate_i = path4+i+"/"
    tstate_yr = os.listdir(tstate_i)
    for j in tstate_yr:
        tstate_yr_sum = tstate_i+j+"/"  
        tstate_qtr = os.listdir(tstate_yr_sum)
        for k in tstate_qtr:
            tstate_k = tstate_yr_sum+k
            tdatas = open(tstate_k,"r")
            tform = json.load(tdatas)
            for z in tform["data"]["districts"]:
                
               
                district =z["entityName"]
                #print(district)
                #print(transaction_count)
                #print(transaction_amount)
                for x in tform["data"]["pincodes"]:
                    pincode =x['entityName']
                    transaction_count = z['metric']['count']
                    transaction_amount = z["metric"]["amount"]
                    file4["State"].append(i)
                    file4["Year"].append(j)
                    file4["Quater"].append(int(k.strip(".json")))
                    file4["District"].append(district)
                    file4["Pincode"].append(pincode)
                    file4["Transaction_count"].append(transaction_count)
                    file4["Transaction_amount"].append(transaction_amount)
               
           
top_transactions = pd.DataFrame(file4)
top_transactions

,State,Year,Quater,District,Transaction_count,Transaction_amount,Pincode
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07,744101
1,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07,744103
2,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07,744102
3,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07,744105
4,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07,744104
...,...,...,...,...,...,...,...
59094,west-bengal,2022,4,paschim medinipur,12768161,2.521681e+10,722101
59095,west-bengal,2022,4,paschim medinipur,12768161,2.521681e+10,700135
59096,west-bengal,2022,4,paschim medinipur,12768161,2.521681e+10,732101
59097,west-bengal,2022,4,paschim medinipur,12768161,2.521681e+10,700091


In [28]:
tt_df = top_transactions.copy()
tt_df["Transaction_amount"]= tt_df["Transaction_amount"].astype(int)
tt_df["Year"] = pd.to_datetime(tt_df["Year"])
tt_df["Year"] = tt_df["Year"].dt.year
tt_df.head()

c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


,State,Year,Quater,District,Transaction_count,Transaction_amount,Pincode
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,744101
1,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,744103
2,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,744102
3,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,744105
4,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249,744104


In [29]:
#top_user
path5= "C:/Users/Home/Documents/Downloads/pulse-master/pulse-master/data/top/user/country/india/state/"
states_list5 = os.listdir(path5)
states_list5

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']

In [30]:
file5 = {"State":[],"Year":[],"Quater":[],"District":[],"RegisteredUser":[],"Pincode":[]}
for i in states_list5:
    ustate_i = path5+i+"/"
    ustate_yr = os.listdir(ustate_i)
    for j in tstate_yr:
        ustate_yr_sum = ustate_i+j+"/"  
        ustate_qtr = os.listdir(ustate_yr_sum)
        for k in tstate_qtr:
            ustate_k = ustate_yr_sum+k
            udatas = open(ustate_k,"r")
            uform = json.load(udatas)
            for z in uform["data"]["districts"]:
             district =z["name"]
             registeredUsers = z["registeredUsers"]
             for x in uform["data"]["pincodes"]:
                pincode = x['name']
                file5["State"].append(i)
                file5["Year"].append(j)
                file5["Quater"].append(int(k.strip(".json")))
                file5["District"].append(district)
                file5["RegisteredUser"].append(registeredUsers)
                file5["Pincode"].append(pincode)
            
top_users = pd.DataFrame(file5)
top_users

,State,Year,Quater,District,RegisteredUser,Pincode
0,andaman-&-nicobar-islands,2018,1,south andaman,5846,744103
1,andaman-&-nicobar-islands,2018,1,south andaman,5846,744101
2,andaman-&-nicobar-islands,2018,1,south andaman,5846,744105
3,andaman-&-nicobar-islands,2018,1,south andaman,5846,744102
4,andaman-&-nicobar-islands,2018,1,south andaman,5846,744104
...,...,...,...,...,...,...
59095,west-bengal,2022,4,purba bardhaman,1119310,700015
59096,west-bengal,2022,4,purba bardhaman,1119310,742304
59097,west-bengal,2022,4,purba bardhaman,1119310,721101
59098,west-bengal,2022,4,purba bardhaman,1119310,700091


In [31]:
tu_df = top_users.copy()
#tu.info()
tu_df["Year"] = pd.to_datetime(tu_df["Year"])
tu_df["Year"] = tu_df["Year"].dt.year 
#tu.info()
tu_df.head()

,State,Year,Quater,District,RegisteredUser,Pincode
0,andaman-&-nicobar-islands,2018,1,south andaman,5846,744103
1,andaman-&-nicobar-islands,2018,1,south andaman,5846,744101
2,andaman-&-nicobar-islands,2018,1,south andaman,5846,744105
3,andaman-&-nicobar-islands,2018,1,south andaman,5846,744102
4,andaman-&-nicobar-islands,2018,1,south andaman,5846,744104


In [32]:
#list of DataFrames created

list_of_df = [x for x in globals() if isinstance(globals()[x],pd.core.frame.DataFrame)and x.endswith('_df')]
list_of_df

['ph1_df', 'ud_df', 'mt_df', 'dt_df', 'tt_df', 'tu_df']

In [ ]:
#Exporting Dataframes to csv 

ph1_df.to_csv("agg_tran.csv",index=False)
ud_df.to_csv("agg_user.csv",index=False)
mt_df.to_csv("map_tran.csv",index=False)
dt_df.to_csv("map_user.csv",index=False)
tt_df.to_csv("top_tran.csv",index=False)
tu_df.to_csv("top_user.csv",index=False)

In [34]:
#connect to sql, create and insert table
mydb= mysql.connector.connect ( host="127.0.0.1",user="root",password="BH7@ravi$",)
mycursor =mydb.cursor()
mycursor.execute("use phonepedata")

In [35]:
mycursor.execute("create table agg_trans(State varchar(255),Year int,Quater int,Transaction_type varchar(255),Transaction_count int,Transaction_amount int)")
for i,row in ph1_df.iterrows():
    sql = "INSERT INTO agg_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [36]:
mycursor.execute("create table agg_user(State varchar(255),Year int,Quater int,Brand varchar(255),Brand_count int,Brand_percentage float)")
for i,row in ud_df.iterrows():
    sql = "INSERT INTO agg_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [39]:
mycursor.execute("create table map_trans(State varchar(255),Year int,Quater int,District varchar(255),Transaction_count int,Transaction_amount int)")
for i,row in mt_df.iterrows():
    sql = "INSERT INTO map_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [40]:
mycursor.execute("create table map_user(State varchar(255),Year int,Quater int,District varchar(255),Registered_user int,App_opening int)")
for i,row in dt_df.iterrows():
    sql = "INSERT INTO map_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [41]:
mycursor.execute("create table top_trans(State varchar(255),Year int,Quater int,District varchar(255),Transaction_count int,Transaction_amount int,Pincode int)")
for i,row in tt_df.iterrows():
    sql = "INSERT INTO top_trans VALUES (%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [46]:
mycursor.execute("create table top_user(State varchar(255),Year int,Quater int,District varchar(255),RegisteredUser int,Pincode int)")
for i,row in tu_df.iterrows():
    sql = "INSERT INTO top_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [ ]:
#query
# mycursor.execute(f"select State,sum(Transaction_count)as Total_transaction, sum(Transaction_amount)as Total_amount
                 from agg_trans,where year={yr} and quater={qtr}group by State order by Total desc limit 10")